References: 
* [Haskell Documentation For Network.HTTP.Client](https://hackage.haskell.org/package/http-client-0.7.13.1/docs/Network-HTTP-Client.html)
* [NGFS API Documentation](https://documenter.getpostman.com/view/1057691/SWE6Zcmd#7d7da29b-a02a-4648-adc3-57f2492015c2)
* [stackoverflow comment explaining how to add environment variables to a custom jupyter kernel](https://stackoverflow.com/a/53595397)

In [ ]:
:extension OverloadedStrings DeriveGeneric

In [ ]:
import Network.HTTP.Client
import Network.HTTP.Types.Status (statusCode)
import Data.Aeson (object, (.=), encode)
import Data.Text (Text)

In [ ]:
import Network.HTTP.Client.TLS (tlsManagerSettings)

In [ ]:
import qualified Data.ByteString.Lazy as LBS (ByteString, filter, toStrict)

In [ ]:
import qualified Data.ByteString as BS (ByteString, filter)

In [ ]:
import Data.Word8 (_quotedbl)

In [ ]:
import System.Environment (getEnv)

In [ ]:
username <- getEnv "IIASA_USERNAME"
password <- getEnv "IIASA_PASSWORD"
application <- getEnv "IIASA_APPLICATION"

In [ ]:
manager <- newManager tlsManagerSettings

-- Create the request
let requestObject = object [ "username" .= username
                           , "password" .= password
                           , "application" .= application ]

initialRequest <- parseRequest "https://db1.ene.iiasa.ac.at/EneAuth/config/v1/login"
let loginReqHeaders = [("Content-Type", "application/json")] -- Accept not needed
    request = initialRequest { method = "POST"
                             , requestBody = RequestBodyLBS $ encode requestObject
                             , requestHeaders = loginReqHeaders } 

response <- httpLbs request manager
putStrLn $ "The status code was: " ++ (show $ statusCode $ responseStatus response)
let token = LBS.toStrict $ LBS.filter (/=_quotedbl) (responseBody response) --  :: LBS.ByteString

In [ ]:
-- TODO: Remove this so as not to show the token
token 

In [ ]:
let phase4baseurl = "https://db1.ene.iiasa.ac.at/ngfs-api/rest/v2.1/"

In [ ]:
import GHC.Generics
import Data.Aeson

## Models Endpoint

In [ ]:
data NgfsModel = NgfsModel { ngfsModel_id :: Int
                           , ngfsModel_name :: Text }
  deriving (Eq, Show, Generic)

instance FromJSON NgfsModel where 
  parseJSON = genericParseJSON (defaultOptions { fieldLabelModifier = Prelude.drop (Prelude.length "ngfsModel_") })


In [ ]:
initialRequest <- parseRequest (phase4baseurl ++ "models")
let request = applyBearerAuth token initialRequest
modelsResponse <- httpLbs request manager
putStrLn $ "The status code was: " ++ (show $ statusCode $ responseStatus response)

In [ ]:
print $ responseBody modelsResponse

In [ ]:
let ngfsModelsE = eitherDecode (responseBody modelsResponse) :: Either String [NgfsModel]

In [ ]:
ngfsModelsE

In [ ]:
ngfsModelsE >>= return . map ngfsModel_id

In [ ]:
either (const []) id ngfsModelsE

## Scenarios Endpoint

In [ ]:
scenariosRequest <- applyBearerAuth token <$> parseRequest (phase4baseurl ++ "scenarios")
scenariosResponse <- httpLbs scenariosRequest manager
putStrLn $ "The status code was: " ++ (show $ statusCode $ responseStatus scenariosResponse)

In [ ]:
print $ responseBody scenariosResponse

In [ ]:
data NgfsScenarios = NgfsScenarios { ngfsScenarios_id :: Int
                                   , ngfsScenarios_name :: Text }
  deriving (Eq, Show, Generic)

instance FromJSON NgfsScenarios where 
  parseJSON = genericParseJSON (defaultOptions { fieldLabelModifier = Prelude.drop (Prelude.length "ngfsScenarios_") })


In [ ]:
let ngfsScenariosE = eitherDecode (responseBody scenariosResponse) :: Either String [NgfsScenarios]
ngfsScenariosE

## Docs Endpoint

In [ ]:
-- Create the request
let docsRequestObject = object [ "keys" .= [ "/metadata/category"
                                           , "/metadata/project"
                                           , "/scenarios/61" ] ]

docsRequestInit <-  parseRequest (phase4baseurl ++ "docs")


let docsReqHeaders = [ ("Accept", "application/json"), ("Content-Type", "application/json")]
    docsRequest = applyBearerAuth token docsRequestInit { method = "POST"
                                                        , requestBody = RequestBodyLBS $ encode docsRequestObject
                                                        , requestHeaders = docsReqHeaders } 

docsResponse <- httpLbs docsRequest manager
putStrLn $ "The status code was: " ++ (show $ statusCode $ responseStatus docsResponse)
print $ responseBody docsResponse

In [ ]:
data NgfsDocs = NgfsDocs { ngfsDocs_key :: Text
                         , ngfsDocs_description :: Text }
  deriving (Eq, Show, Generic)

instance FromJSON NgfsDocs where 
  parseJSON = genericParseJSON (defaultOptions { fieldLabelModifier = Prelude.drop (Prelude.length "NgfsDocs_") })


In [ ]:
let ngfsDocsE = eitherDecode (responseBody docsResponse) :: Either String [NgfsDocs]
ngfsDocsE

## TS Endpoint

In [ ]:
tsRequest <- applyBearerAuth token <$> parseRequest (phase4baseurl ++ "ts")
tsResponse <- httpLbs tsRequest manager
putStrLn $ "The status code was: " ++ (show $ statusCode $ responseStatus tsResponse)

In [ ]:
-- printing here consumes too many resources
-- print $ responseBody tsResponse

In [ ]:
data NgfsTs = NgfsTs { ngfsTs_id :: Int
                     , ngfsTs_variable :: Text
                     , ngfsTs_unitid :: Int
                     , ngfsTs_unit :: Text
                     , ngfsTs_runIds :: [Int]
                     , ngfsTs_regions :: [Int] }
  deriving (Eq, Show, Generic)

instance FromJSON NgfsTs where 
  parseJSON = genericParseJSON (defaultOptions { fieldLabelModifier = Prelude.drop (Prelude.length "ngfsTs_") })


In [ ]:
let ngfsTsE = eitherDecode (responseBody tsResponse) :: Either String [NgfsTs]

In [ ]:
fmap (map ngfsTs_id) ngfsTsE